Импортируем все нужные библиотеки

In [77]:
import vk_api
import pandas as pd
import numpy as np
import time
import datetime

Задаём токен в переменную

In [78]:
token = ''

Авторизация

In [ ]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

Функция для сбора данных о подписчиках нескольких групп, примнимает на вход список групп

In [96]:
def get_subscriber_info(groups):
    subscribers_list = []
    
    for group_id in groups:
        #Количество подписчиков в группе 
        group_members_count = vk.groups.getMembers(group_id=group_id)['count']
        
        group_name = vk.groups.getById(group_id=group_id)[0]['name']

        # Размер выборки в одном запросе
        batch_size = 1000
        
        # Делим общее количество пользователей на отрезки по размеру выборки и поочередно собираем данные
        for i in range(0, group_members_count, batch_size):
            #Отбираем нужные колонки
            response = vk.groups.getMembers(group_id=group_id, offset=i, count=batch_size, fields=['bdate',
                                                                          'sex',
                                                                          'city', 
                                                                          'education',
                                                                          'universities',
                                                                          'occupation'
                                                                          ])
            #Задаём имя
            items = response['items']

            #Проходимся по каждой графе каждого пользователя и отбираем данные
            for item in items:
                user = {
                    'id': item['id'],
                    'sex': item['sex'],
                    'bdate': item.get('bdate', np.nan),
                    'city': item.get('city', {'title': np.nan})['title'],
                    'university': item.get('university_name', np.nan),
                    'graduation': item.get('graduation', np.nan),
                    'is_closed': item.get('is_closed'),
                    'group_name': group_name
                    }
                    
                #Проверка некоторых колонок
                if user['graduation'] == 0:
                    user['graduation'] = np.nan
                if 'occupation' in item:
                    if item['occupation']['type'] == 'university':
                        user['university'] = item['occupation']['name']
                        if 'graduate_year' in item['occupation']:
                            user['graduation'] = item['occupation']['graduate_year']
                if user['university'] == '':
                    user['university'] = np.nan


                #Добавляем информацию по пользователю в список
                subscribers_list.append(user)
            
            #Таймер, чтобы не превысить кол-во запросов API
            time.sleep(0.5)
        
        #Информация по группе
        print(f'В группе "{group_name}" собраны данные о {group_members_count} подписчиках')
        
    result = pd.DataFrame(subscribers_list)
    
    print(f'Готово! Собранны данные о {len(result)} пользователях')
    
    return result.reset_index(drop=True)

Задаём в переменную список групп и выполняем функцию, результат записываем в переменную

In [97]:
groups = [169861409, 148664796, 87585181, 89513564, 89975111, 44536327, 38807981, 85788343, 35863861]
final = get_subscriber_info(groups)

В группе "Библиотеки Приморского района" собраны данные о 3336 подписчиках
В группе "Библиотека Салтыкова-Щедрина" собраны данные о 1987 подписчиках
В группе "Библиотека  на  Торжковской" собраны данные о 1432 подписчиках
В группе "Библиотека № 3 Приморского района С.-Петербурга" собраны данные о 169 подписчиках
В группе "Библиотека №4 в Ольгино" собраны данные о 457 подписчиках
В группе "Библиотека Лисий Нос" собраны данные о 660 подписчиках
В группе "Библиотека №7  Приморского района" собраны данные о 1124 подписчиках
В группе "Библиотека№8 Приморский район" собраны данные о 593 подписчиках
В группе "Библиотека № 9 Приморский район" собраны данные о 1661 подписчиках
Готово! Собранны данные о 10983 пользователях


In [98]:
final

,id,sex,bdate,city,university,graduation,is_closed,group_name
0,1378,1,2.2,Санкт-Петербург,СПбГУ,2010.0,True,Библиотеки Приморского района
1,2006,1,18.8.1984,Санкт-Петербург,СПбГУКИ,2006.0,True,Библиотеки Приморского района
2,3269,2,26.12.1986,Санкт-Петербург,NaN,NaN,True,Библиотеки Приморского района
3,3326,2,6.5.1986,Санкт-Петербург,СПбГЭТУ (ЛЭТИ),2009.0,False,Библиотеки Приморского района
4,3357,1,13.8,Санкт-Петербург,NaN,NaN,False,Библиотеки Приморского района
...,...,...,...,...,...,...,...,...
10978,762024904,1,21.5.1956,NaN,NaN,NaN,False,Библиотека № 9 Приморский район
10979,762560681,1,7.7.2003,NaN,NaN,NaN,True,Библиотека № 9 Приморский район
10980,763498438,1,27.10.1973,Санкт-Петербург,NaN,NaN,False,Библиотека № 9 Приморский район
10981,772095981,1,14.6,Санкт-Петербург,ЧГИК (бывш. ЧГАКИ),2003.0,False,Библиотека № 9 Приморский район


Сохраняем датасет

In [41]:
final.to_csv(f'subscriber_data_{str(datetime.datetime.today()).split()[0]}.csv')

Функция для сбора информации о группах пользователя, на вход принимает предыдущий датасет

In [99]:
def get_users_subscriptions(data):
    
    #Удаляем пользователей, которые подписаны сразу на несколько групп библиотек
    data = data.drop_duplicates(subset='id')
    
    #Создаём список из ID подписчиков
    ids = data.id.tolist()
    
    #Задаём результат
    result = pd.DataFrame(columns=['user_id', 
                                   'subscription_id',
                                   'name',
                                   'tag'
                                  ])
    
    #Счётчик
    user_counter = 0
    
    #Проходимся по списку ID
    for user_id in ids:
        
        #Проверяем закрыт ли аккаунт
        if not data.query('id == @user_id')['is_closed'].values[0]:
            
            #Проверяем скрыты ли подписки
            try: 
                #Отбираем информацию о подписках
                response = vk.users.getSubscriptions(user_id=user_id, extended=1, count=200, fields='activity')

                items = response['items']

                users_groups = []

                user_counter += 1

                for item in items:
                    if item['type'] == 'page':
                        group = {
                            'user_id': user_id,
                            'subscription_id': item.get('id'),
                            'name': item.get('name'),
                            'tag': item.get('activity', np.nan)
                        }

                    #Сохраняем в список
                    users_groups.append(group)
                    
                df = pd.DataFrame(users_groups)

                result = pd.concat([result, df])

                print(f'Данные по пользователю {user_counter} из {len(ids)} с ID {user_id} записаны.')
            except ApiError:
                print(f'Профиль пользователя {user_counter} с ID {user_id} закрыт')
            
        else:
            user_counter += 1
            print(f'Профиль пользователя {user_counter} с ID {user_id} закрыт')
    
    print('Готово')
    
    return result   

Применяем функцию

In [100]:
data = get_users_subscriptions(final)

Профиль пользователя 1 с ID 1378 закрыт
Профиль пользователя 2 с ID 2006 закрыт
Профиль пользователя 3 с ID 3269 закрыт
Данные по пользователю 4 из 9280 с ID 3326 записаны.
Данные по пользователю 5 из 9280 с ID 3357 записаны.
Данные по пользователю 6 из 9280 с ID 4074 записаны.
Профиль пользователя 7 с ID 4538 закрыт
Профиль пользователя 8 с ID 5615 закрыт
Данные по пользователю 9 из 9280 с ID 5812 записаны.
Данные по пользователю 10 из 9280 с ID 8412 записаны.
Данные по пользователю 11 из 9280 с ID 13138 записаны.
Профиль пользователя 12 с ID 13490 закрыт
Профиль пользователя 13 с ID 15489 закрыт
Данные по пользователю 14 из 9280 с ID 15939 записаны.
Данные по пользователю 15 из 9280 с ID 16668 записаны.
Данные по пользователю 16 из 9280 с ID 23311 записаны.
Данные по пользователю 17 из 9280 с ID 23320 записаны.
Данные по пользователю 18 из 9280 с ID 23417 записаны.
Данные по пользователю 19 из 9280 с ID 24254 записаны.
Профиль пользователя 20 с ID 24876 закрыт
Данные по пользователю 

Данные по пользователю 162 из 9280 с ID 267570 записаны.
Данные по пользователю 163 из 9280 с ID 268009 записаны.
Данные по пользователю 164 из 9280 с ID 269767 записаны.
Данные по пользователю 165 из 9280 с ID 275188 записаны.
Профиль пользователя 166 с ID 276032 закрыт
Данные по пользователю 167 из 9280 с ID 296895 записаны.
Данные по пользователю 168 из 9280 с ID 300395 записаны.
Данные по пользователю 169 из 9280 с ID 315154 записаны.
Данные по пользователю 170 из 9280 с ID 316054 записаны.
Профиль пользователя 171 с ID 316158 закрыт
Данные по пользователю 172 из 9280 с ID 316833 записаны.
Профиль пользователя 173 с ID 317629 закрыт
Профиль пользователя 174 с ID 317648 закрыт
Данные по пользователю 175 из 9280 с ID 318208 записаны.
Данные по пользователю 176 из 9280 с ID 318446 записаны.
Профиль пользователя 177 с ID 318677 закрыт
Данные по пользователю 178 из 9280 с ID 322326 записаны.
Данные по пользователю 179 из 9280 с ID 322520 записаны.
Профиль пользователя 180 с ID 328448 за

Данные по пользователю 322 из 9280 с ID 767808 записаны.
Профиль пользователя 323 с ID 771347 закрыт
Данные по пользователю 324 из 9280 с ID 771500 записаны.
Профиль пользователя 325 с ID 771810 закрыт
Данные по пользователю 326 из 9280 с ID 772002 записаны.
Профиль пользователя 327 с ID 777182 закрыт
Профиль пользователя 328 с ID 781483 закрыт
Профиль пользователя 329 с ID 784045 закрыт
Данные по пользователю 330 из 9280 с ID 786400 записаны.
Данные по пользователю 331 из 9280 с ID 793262 записаны.
Данные по пользователю 332 из 9280 с ID 793484 записаны.
Профиль пользователя 333 с ID 794884 закрыт
Профиль пользователя 334 с ID 795317 закрыт
Профиль пользователя 335 с ID 796474 закрыт
Данные по пользователю 336 из 9280 с ID 796582 записаны.
Профиль пользователя 337 с ID 801960 закрыт
Данные по пользователю 338 из 9280 с ID 808177 записаны.
Профиль пользователя 339 с ID 820272 закрыт
Профиль пользователя 340 с ID 822080 закрыт
Профиль пользователя 341 с ID 823054 закрыт
Профиль пользова

Данные по пользователю 482 из 9280 с ID 1355572 записаны.
Профиль пользователя 483 с ID 1360325 закрыт
Профиль пользователя 484 с ID 1370492 закрыт
Данные по пользователю 485 из 9280 с ID 1385278 записаны.
Профиль пользователя 486 с ID 1389344 закрыт
Данные по пользователю 487 из 9280 с ID 1395789 записаны.
Данные по пользователю 488 из 9280 с ID 1403079 записаны.
Профиль пользователя 489 с ID 1409539 закрыт
Данные по пользователю 490 из 9280 с ID 1409722 записаны.
Профиль пользователя 491 с ID 1415494 закрыт
Данные по пользователю 492 из 9280 с ID 1423383 записаны.
Данные по пользователю 493 из 9280 с ID 1426366 записаны.
Профиль пользователя 494 с ID 1429314 закрыт
Данные по пользователю 495 из 9280 с ID 1430631 записаны.
Профиль пользователя 496 с ID 1432431 закрыт
Данные по пользователю 497 из 9280 с ID 1439622 записаны.
Профиль пользователя 498 с ID 1440455 закрыт
Данные по пользователю 499 из 9280 с ID 1441660 записаны.
Данные по пользователю 500 из 9280 с ID 1450033 записаны.
Пр

Данные по пользователю 638 из 9280 с ID 2409301 записаны.
Профиль пользователя 639 с ID 2413773 закрыт
Данные по пользователю 640 из 9280 с ID 2433043 записаны.
Профиль пользователя 641 с ID 2441039 закрыт
Данные по пользователю 642 из 9280 с ID 2452410 записаны.
Данные по пользователю 643 из 9280 с ID 2459632 записаны.
Данные по пользователю 644 из 9280 с ID 2464447 записаны.
Данные по пользователю 645 из 9280 с ID 2471950 записаны.
Профиль пользователя 646 с ID 2486869 закрыт
Данные по пользователю 647 из 9280 с ID 2490117 записаны.
Профиль пользователя 648 с ID 2492706 закрыт
Данные по пользователю 649 из 9280 с ID 2503577 записаны.
Профиль пользователя 650 с ID 2530608 закрыт
Данные по пользователю 651 из 9280 с ID 2531902 записаны.
Данные по пользователю 652 из 9280 с ID 2538609 записаны.
Данные по пользователю 653 из 9280 с ID 2543997 записаны.
Данные по пользователю 654 из 9280 с ID 2551593 записаны.
Данные по пользователю 655 из 9280 с ID 2561784 записаны.
Данные по пользовател

...

Данные по пользователю 8566 из 9280 с ID 6518423 записаны.
Данные по пользователю 8567 из 9280 с ID 6575820 записаны.
Данные по пользователю 8568 из 9280 с ID 6646854 записаны.
Профиль пользователя 8569 с ID 6711854 закрыт
Профиль пользователя 8570 с ID 6755201 закрыт
Данные по пользователю 8571 из 9280 с ID 6756499 записаны.
Данные по пользователю 8572 из 9280 с ID 6758998 записаны.
Профиль пользователя 8573 с ID 6808838 закрыт
Профиль пользователя 8574 с ID 6826537 закрыт
Данные по пользователю 8575 из 9280 с ID 6877335 записаны.
Профиль пользователя 8576 с ID 6886721 закрыт
Профиль пользователя 8577 с ID 7041928 закрыт
Данные по пользователю 8578 из 9280 с ID 7090293 записаны.
Данные по пользователю 8579 из 9280 с ID 7107453 записаны.
Профиль пользователя 8580 с ID 7175863 закрыт
Данные по пользователю 8581 из 9280 с ID 7219736 записаны.
Профиль пользователя 8582 с ID 7307914 закрыт
Данные по пользователю 8583 из 9280 с ID 7317181 записаны.
Профиль пользователя 8584 с ID 7339182 зак

Данные по пользователю 8716 из 9280 с ID 20358432 записаны.
Данные по пользователю 8717 из 9280 с ID 20468049 записаны.
Профиль пользователя 8718 с ID 20661269 закрыт
Данные по пользователю 8719 из 9280 с ID 20715860 записаны.
Данные по пользователю 8720 из 9280 с ID 21160034 записаны.
Данные по пользователю 8721 из 9280 с ID 21358723 записаны.
Данные по пользователю 8722 из 9280 с ID 21784503 записаны.
Данные по пользователю 8723 из 9280 с ID 22183427 записаны.
Профиль пользователя 8724 с ID 22294642 закрыт
Данные по пользователю 8725 из 9280 с ID 22319067 записаны.
Профиль пользователя 8726 с ID 22371971 закрыт
Профиль пользователя 8727 с ID 22430241 закрыт
Данные по пользователю 8728 из 9280 с ID 22472605 записаны.
Данные по пользователю 8729 из 9280 с ID 22599543 записаны.
Данные по пользователю 8730 из 9280 с ID 22657095 записаны.
Данные по пользователю 8731 из 9280 с ID 22661742 записаны.
Данные по пользователю 8732 из 9280 с ID 22667821 записаны.
Данные по пользователю 8733 из 9

Данные по пользователю 8865 из 9280 с ID 67152204 записаны.
Профиль пользователя 8866 с ID 67455941 закрыт
Данные по пользователю 8867 из 9280 с ID 67489543 записаны.
Данные по пользователю 8868 из 9280 с ID 68093730 записаны.
Данные по пользователю 8869 из 9280 с ID 69194719 записаны.
Профиль пользователя 8870 с ID 69331383 закрыт
Данные по пользователю 8871 из 9280 с ID 69983632 записаны.
Профиль пользователя 8872 с ID 70039828 закрыт
Данные по пользователю 8873 из 9280 с ID 70626418 записаны.
Профиль пользователя 8874 с ID 71533528 закрыт
Профиль пользователя 8875 с ID 73422905 закрыт
Данные по пользователю 8876 из 9280 с ID 74094482 записаны.
Данные по пользователю 8877 из 9280 с ID 75371927 записаны.
Профиль пользователя 8878 с ID 77367593 закрыт
Профиль пользователя 8879 с ID 77385507 закрыт
Данные по пользователю 8880 из 9280 с ID 78508417 записаны.
Данные по пользователю 8881 из 9280 с ID 78827891 записаны.
Данные по пользователю 8882 из 9280 с ID 78869460 записаны.
Данные по п

Данные по пользователю 9009 из 9280 с ID 174594323 записаны.
Данные по пользователю 9010 из 9280 с ID 174871146 записаны.
Профиль пользователя 9011 с ID 174882351 закрыт
Профиль пользователя 9012 с ID 175033304 закрыт
Данные по пользователю 9013 из 9280 с ID 175241906 записаны.
Профиль пользователя 9014 с ID 175491770 закрыт
Профиль пользователя 9015 с ID 176517050 закрыт
Данные по пользователю 9016 из 9280 с ID 176685785 записаны.
Данные по пользователю 9017 из 9280 с ID 178509455 записаны.
Данные по пользователю 9018 из 9280 с ID 178652857 записаны.
Профиль пользователя 9019 с ID 179135572 закрыт
Профиль пользователя 9020 с ID 179849026 закрыт
Данные по пользователю 9021 из 9280 с ID 180162162 записаны.
Данные по пользователю 9022 из 9280 с ID 181741460 записаны.
Данные по пользователю 9023 из 9280 с ID 182227139 записаны.
Данные по пользователю 9024 из 9280 с ID 182275730 записаны.
Данные по пользователю 9025 из 9280 с ID 183505621 записаны.
Данные по пользователю 9026 из 9280 с ID 

Данные по пользователю 9154 из 9280 с ID 332984150 записаны.
Данные по пользователю 9155 из 9280 с ID 334688494 записаны.
Профиль пользователя 9156 с ID 335128448 закрыт
Данные по пользователю 9157 из 9280 с ID 335644432 записаны.
Данные по пользователю 9158 из 9280 с ID 337974869 записаны.
Данные по пользователю 9159 из 9280 с ID 342056748 записаны.
Данные по пользователю 9160 из 9280 с ID 342665292 записаны.
Профиль пользователя 9161 с ID 343005819 закрыт
Профиль пользователя 9162 с ID 346170968 закрыт
Данные по пользователю 9163 из 9280 с ID 346958988 записаны.
Данные по пользователю 9164 из 9280 с ID 347102252 записаны.
Данные по пользователю 9165 из 9280 с ID 347999490 записаны.
Данные по пользователю 9166 из 9280 с ID 348807533 записаны.
Данные по пользователю 9167 из 9280 с ID 350203918 записаны.
Профиль пользователя 9168 с ID 350360829 закрыт
Данные по пользователю 9169 из 9280 с ID 353449290 записаны.
Данные по пользователю 9170 из 9280 с ID 355410054 записаны.
Данные по польз

In [104]:
data.to_csv(f'subscriptions_info_{str(datetime.datetime.today()).split()[0]}.csv')